<a href="https://colab.research.google.com/github/shumshersubashgautam/Pyspark-NLP/blob/main/Detect_and_identify_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

Spark NLP version 4.4.1
Apache Spark version: 3.2.3


In [43]:
from sparknlp.pretrained import PretrainedPipeline

In [44]:
# Download a pre-trained pipeline by name and language
language_detector_pipeline = PretrainedPipeline('detect_language_21', lang='xx')

# Depending on the language (how similar the characters are), the LanguageDetectorDL works
# best with text longer than 140 characters
language_detector_pipeline.annotate("«Нападение на 13-й участок»")

detect_language_21 download started this may take some time.
Approx size to download 7.7 MB
[OK!]


{'document': ['«Нападение на 13-й участок»'],
 'sentence': ['«Нападение на 13-й участок»'],
 'language': ['bg']}

In [45]:
# LanguageDetectorDL
# Pre-trained Models
from sparknlp.base import *
from sparknlp.annotator import *

In [46]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

language_detector = LanguageDetectorDL.pretrained("ld_wiki_tatoeba_cnn_21")\
.setInputCols(["document"])\
.setOutputCol("lang")\
.setThreshold(0.8)\
.setCoalesceSentences(True)

languagePipeline = Pipeline(stages=[
 documentAssembler,
 language_detector
])

ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[OK!]


In [47]:
test_df = spark.createDataFrame([
  ['Spark NLP is an open-source text processing library for advanced natural language processing for the Python, Java and Scala programming languages.'], 
  ['Spark NLP est une bibliothèque de traitement de texte open source pour le traitement avancé du langage naturel pour les langages de programmation Python, Java et Scala.']]
).toDF("text")

results = languagePipeline.fit(test_df).transform(test_df)

In [48]:
results.select("lang.result").show()

+------+
|result|
+------+
|  [en]|
|  [fr]|
+------+



In [49]:
# probabilities for other languages
results.select("lang.metadata").show(2, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                                                                                                                                                                                                                                                                                                                                             |
+-------------------------------------------------------------------------------------------------------------------------------------------------------